In [1]:
import pandas as pd
import numpy as np

# Mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Mostrar todas las filas (si quieres limitarlo, usa por ejemplo 200)
pd.set_option('display.max_rows', None)

# Ampliar el ancho de cada columna para ver bien el texto
pd.set_option('display.max_colwidth', None)

# Ajustar el ancho de la tabla en consola
pd.set_option('display.width', 0)

In [3]:
import xmlrpc.client
import pandas as pd

# --- Conexión ---
url = "https://donsson.com"
db = "Donsson_produccion"
username = "juan.cano@donsson.com"
password = "1000285668"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# --- 1️⃣ Órdenes de producción terminadas desde 2023 ---
producciones = models.execute_kw(
    db, uid, password,
    "mrp.production", "search_read",
    [[("state", "=", "done"), ("date_finished", ">=", "2023-01-01")]],
    {"fields": ["id", "name", "date_finished", "product_id", "product_qty"]}
)
df_mo = pd.DataFrame(producciones)
print(f"Órdenes de producción encontradas: {len(df_mo)}")

# --- Limpiar producto terminado ---
if "product_id" in df_mo.columns:
    df_mo["producto_terminado_id"] = df_mo["product_id"].apply(lambda x: x[0] if isinstance(x, list) else None)
    df_mo["producto_terminado"] = df_mo["product_id"].apply(lambda x: x[1] if isinstance(x, list) else None)

# --- 2️⃣ Apuntes contables de Materias Primas ---
account_lines = models.execute_kw(
    db, uid, password,
    "account.move.line", "search_read",
    [[("account_id.code", "=", "1405051002"), ("ref", "!=", False)]],
    {"fields": ["ref", "date", "product_id", "quantity", "credit", "debit", "name", "product_uom_id"]}
)
df_acc = pd.DataFrame(account_lines)
print(f"Apuntes contables descargados: {len(df_acc)}")

# --- 3️⃣ Normalizar productos y unidades ---
if "product_id" in df_acc.columns:
    df_acc["product_id_int"] = df_acc["product_id"].apply(lambda x: x[0] if isinstance(x, list) else None)
    df_acc["producto"] = df_acc["product_id"].apply(lambda x: x[1] if isinstance(x, list) else None)
if "product_uom_id" in df_acc.columns:
    df_acc["unidad"] = df_acc["product_uom_id"].apply(lambda x: x[1] if isinstance(x, list) else None)

# --- 4️⃣ Unir con MOs ---
df_acc["MO"] = df_acc["ref"].fillna(df_acc["name"]).astype(str)
df_acc = df_acc[df_acc["MO"].isin(df_mo["name"])]

df_final = df_acc.merge(
    df_mo[["name", "date_finished", "producto_terminado"]],
    left_on="MO", right_on="name", how="left"
)

# --- 5️⃣ Calcular costo total ---
df_final["costo_total"] = df_final["credit"].fillna(0) - df_final["debit"].fillna(0)

# --- 6️⃣ Seleccionar columnas limpias ---
df_resultado = df_final[[
    "MO", "date", "producto", "quantity", "unidad",
    "costo_total", "date_finished", "producto_terminado"
]]

# --- 7️⃣ Exportar ---
df_resultado.to_excel("consumo_materias_primas_2023_2025.xlsx", index=False)
print("✅ Archivo generado: consumo_materias_primas_2023_2025.xlsx")



Órdenes de producción encontradas: 4625


KeyboardInterrupt: 

In [ ]:
df_resultado .sample(10)